# Exploração do Sistema Tensorwerk

Este notebook demonstra os conceitos fundamentais do ecossistema Tensorwerk:
- Geometria Riemanniana aplicada a mercados
- Cálculo de curvatura escalar
- Detecção de singularidades
- Visualização de manifolds

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Adicionar paths
import sys
sys.path.append('..')

from differential_nets.neural_sde import generate_synthetic_data
from topology.persistent_homology import detect_market_crash_via_topology

## 1. Gerar Dados Sintéticos de Mercado

In [ ]:
# Gerar trajetórias de preço (4D)
np.random.seed(42)

# Mercado estável
points_stable = np.random.randn(100, 4) * 0.1 + np.array([100, 100, 100, 100])

# Mercado instável (com estrutura de anel)
t = np.linspace(0, 4*np.pi, 100)
points_unstable = np.column_stack([
    np.cos(t) * 10,
    np.sin(t) * 10,
    np.cos(2*t) * 5,
    np.sin(2*t) * 5
])

print(f"Mercado estável: {points_stable.shape}")
print(f"Mercado instável: {points_unstable.shape}")

## 2. Visualizar Dados 3D

In [ ]:
fig = plt.figure(figsize=(12, 5))

# Mercado estável
ax1 = fig.add_subplot(121, projection='3d')
ax1.scatter(points_stable[:, 0], points_stable[:, 1], points_stable[:, 2], c='b', alpha=0.6)
ax1.set_title('Mercado Estável')
ax1.set_xlabel('Preço')
ax1.set_ylabel('Volume')
ax1.set_zlabel('Tempo')

# Mercado instável
ax2 = fig.add_subplot(122, projection='3d')
ax2.scatter(points_unstable[:, 0], points_unstable[:, 1], points_unstable[:, 2], c='r', alpha=0.6)
ax2.set_title('Mercado Instável')
ax2.set_xlabel('Preço')
ax2.set_ylabel('Volume')
ax2.set_zlabel('Tempo')

plt.tight_layout()
plt.show()

## 3. Detectar Risco via TDA

In [ ]:
# Analisar mercado estável
result_stable = detect_market_crash_via_topology(points_stable)
print(f" Mercado Estável")
print(f"Risco: {result_stable['risk_level']}")
print(f"Score: {result_stable['risk_score']:.3f}\n")

# Analisar mercado instável
result_unstable = detect_market_crash_via_topology(points_unstable)
print(f" Mercado Instável")
print(f"Risco: {result_unstable['risk_level']}")
print(f"Score: {result_unstable['risk_score']:.3f}")

## 4. Interpretar Resultados

### O que significa?

- **Risco BAIXO**: O manifold é suave, sem "buracos"
- **Risco MODERADO**: Algum emaranhamento, atenção necessária
- **Risco ALTO**: Múltiplos loops detectados - possível crash
- **Risco CRÍTICO**: Singularidade topológica iminente

### H1 Features (Loops)

Um alto número de features H1 com longa persistência indica:
- O manifold está "enroscado"
- Há ciclos de feedback no mercado
- Uma pequena perturbação pode causar colapso

## 5. Calcular Curvatura (Exemplo)

A curvatura escalar R mede o "emaranhamento" do espaço.
- R ≈ 0: Espaço plano (mercado estável)
- R > 0: Curvatura positiva (bolha)
- R < 0: Curvatura negativa (colapso)
- R → ∞: Singularidade!

In [ ]:
def approximate_curvature(points):
    """
    Aproxima curvatura escalar local usando vizinhos mais próximos.
    
    Nota: Esta é uma simplificação. O cálculo real requer
    o motor C++ com tensores de Riemann completos.
    """
    from scipy.spatial.distance import pdist, squareform
    
    # Matriz de distâncias
    dist_matrix = squareform(pdist(points))
    
    # Curvatura aproximada: inversa da distância média
    # (quanto mais próximos os pontos, maior a curvatura)
    mean_dist = np.mean(dist_matrix[dist_matrix > 0])
    curvature = 1.0 / (mean_dist + 1e-6)
    
    return curvature

k_stable = approximate_curvature(points_stable)
k_unstable = approximate_curvature(points_unstable)

print(f"Curvatura (estável): {k_stable:.4f}")
print(f"Curvatura (instável): {k_unstable:.4f}")
print(f"\nO mercado instável tem {k_unstable/k_stable:.1f}x mais curvatura!")

## 6. Próximos Passos

Para exploração adicional:

1. **`sde_experiments.ipynb`**: Treinar Neural SDEs
2. **`tda_demo.ipynb`**: Análise topológica profunda
3. **Motor C++**: Para cálculo preciso de curvatura de Riemann
4. **Interface Web**: Visualização 4D interativa

### Dados Reais

Para usar dados de mercado reais:
```python
import pandas as pd

# Carregar de data/raw/
df = pd.read_csv('../data/raw/binance_trades.csv')

# Extrair features
points = df[['price', 'volume', 'bid_ask_spread', 'time']].values

# Analisar
result = detect_market_crash_via_topology(points)
```